In [1]:
from models.definitions.resnet_vae import ResnetVAE
from utils.dataloaders.dataloader_cifar10_single import DataLoaderCIFAR10

from torchvision import transforms
import torch
from torch.optim import Adam

import sys
# Add sys path for importing project modules
sys.path.append('../')


In [9]:
config = {
    'model_name': 'VAE',
    'dataset': 'CIFAR10',
    'seed': 4,
    # Model setup 
    'in_chan': 3,
    'latent_dim': 16,
    # Training setup
    'batch_size': 256,
    'num_epochs': 15,
    'learning_rate': 0.0002,
    'latent_dim': 16
}

# DEVICE for Mac for Windows use CUDA
DEVICE = torch.device("mps")
print(DEVICE)

# Set the seed
torch.manual_seed(config['seed'])

mps


In [3]:
transform = [transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]


In [4]:
dataloader = DataLoaderCIFAR10(batch_size=32, transformation=transform)

test_loader = dataloader.get_test_loader()
train_loader = dataloader.get_train_loader()

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Model Initialization
model = ResnetVAE(config['latent_dim'], config['in_chan']).to(DEVICE)

In [11]:
optimizer = Adam(model.parameters(), lr=config['learning_rate'])

def train(model, train_loader, config):
    model.train()
    for epoch in range(config['num_epochs']):
        overall_loss = 0
        for batch_idx, (x, _) in enumerate(train_loader):
            x = x.to(DEVICE)
            optimizer.zero_grad()
            x_hat, mean, log_var = model(x)
            loss = model.loss_function(x, x_hat, mean, log_var)
            overall_loss += loss.item()
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                print("\tEpoch", epoch + 1, "complete!", "\tBatch", batch_idx, "complete!", "\tLoss:", loss.item())
        print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / (batch_idx*config['batch_size']))
    print("Finish!!")




In [12]:
# Check how many parameters in the model
num_params = sum(p.numel() for p in model.parameters())
print('Number of parameters: %d' % num_params)

Number of parameters: 1016019
